# Package List

In [1]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
import pickle

/home/ubuntu2204/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Load

In [2]:
# df = pd.read_csv("../data/서울시대여공구찾기정보.csv", encoding='cp949')
df = pd.read_csv("../data/prep3.csv", encoding='cp949')

In [4]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,11488,11489,11490,11491,11492,11493,11494,11495,11496,11497
구분자(PK),13423,14913,14038,13678,13677,16092,9988,15168,13571,15470,...,3934,5060,5320,5240,5295,5342,863,897,959,975
공구 대분류 코드,4,4,4,4,4,4,4,4,4,4,...,2,2,2,2,2,2,2,2,2,2
공구 대분류,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,...,전동공구,전동공구,전동공구,전동공구,전동공구,전동공구,전동공구,전동공구,전동공구,전동공구
공구 중분류 코드,44,44,44,44,44,44,44,44,44,44,...,22,22,22,22,22,22,22,22,22,22
공구 중분류,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,기타공구,...,해머드릴,해머드릴,해머드릴,해머드릴,해머드릴,해머드릴,해머드릴,해머드릴,해머드릴,해머드릴
공구 이름,고압세척기,(57)레이저거리측정기,2단 사다리,3단사다리,4단사다리,4방향 냉동라쳇,4방향 냉동라쳇,50m 줄자,LCD전자 현미경,LED 작업등,...,햄머드릴,햄머드릴,햄머드릴 1개,햄머드릴 1개,햄머드릴 1개,햄머드릴 1개,햄머드릴세트,햄머드릴세트,햄머드릴세트,햄머드릴세트
과금기준,없음,없음,없음,없음,없음,없음,없음,없음,1일,없음,...,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음
수량,1,1,1,1,1,1,1,2,1,1,...,3,1,1,1,1,1,1,1,2,1
장소 PK,501,172,13214,495,495,13025,13022,455,489,13212,...,331,292,304,301,303,306,104,106,111,112
대여장소명,황학동 공구대여소,우리마을 공구대여소(사당1동),묵1동 공유마을 공유대여소,신당동주민센터 공구도서관,신당동주민센터 공구도서관,삼성1동 주민센터 공구대여소,도곡1동 주민센터,불광2동 공구도서관,회현동 공구대여소,수색동주민센터,...,신정6동 주민센터 공구대여소,오금현대 공구백화점,송파파인타운6단지,문정래미안,거여1단지,위례24단지,가산동 주민센터 공구도서관,독산2동 주민센터 공구도서관,시흥4동 주민센터 공구도서관,시흥5동 주민센터 공구도서관


## Preprocessing

In [4]:
user_info = df.T.to_dict()

# Embedding

In [5]:
with open('../data/encoded_tool_name.pkl', "rb") as f:
    arr_loaded = pickle.load(f)

In [12]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=10,          # 작게 묶기
    # min_samples=10,               # 밀도 추정 완화
    cluster_selection_epsilon=0.5,  # 거리 허용 키우기 (단위는 임베딩에 따라 튜닝 필요)
    # cluster_selection_method='eom', # or 'leaf' 로 바꿔도 좋음
    # metric='euclidean'           # cosine 도 고려 가능
)

cluster_labels = clusterer.fit_predict(arr_loaded)

In [13]:
df['cluster'] = cluster_labels

In [14]:
df['cluster'].value_counts()

cluster
-1      3173
 76      256
 176     234
 101     217
 160     207
        ... 
 202      11
 219      11
 194      10
 152      10
 161      10
Name: count, Length: 227, dtype: int64

In [ ]:
df[df['공구 중분류'].str.contains('사다리')][''].unique()

array(['사다리'], dtype=object)

In [29]:
df[df['공구 중분류'].str.contains('기타')]['공구 중분류'].unique()

array(['기타공구', '기타 생활용품', '기타 일반공구', '기타 전기제품', '기타 전동공구'], dtype=object)

In [21]:
for mid in df['공구 중분류'].unique():
    print("공구 중분류:", mid)
    print("개수 : ", len(df[df['공구 중분류']==mid]['공구 이름'].unique()))
    print(df[df['공구 중분류']==mid]['공구 이름'].unique())
    print("*"*50)
    # break

공구 중분류: 기타공구
개수 :  244
[' 고압세척기 ' '(57)레이저거리측정기' '2단 사다리' '3단사다리' '4단사다리' '4방향 냉동라쳇' '50m 줄자'
 'LCD전자 현미경' 'LED 작업등' 'LED 포토박스' 'LED랜턴' 'led 작업등' 'ㄷ형 타카핀' '가스토치'
 '가열총(열풍기)' '가정용 공구세트' '거리측정기' '거짓말탐지기' '건 타카' '건습식 청소기(집진형)' '건타카'
 '건타카+타카핀' '경광등' '경량몽키' '고무롤러' '고무망치' '고압 세척기' '고압력 쌍발 발펌프(자동차,오토바이,자전거)'
 '공구놀이세트' '공구세트' '공구함' '공구함(내용물 미포함)' '공유우산 ' '관통기' '글루건' '글루건 건' '글루건 심'
 '글루건+막대본드' '글루건, 핫멜트봉' '글루스틱' '기리세트(계양용)' '납 인두기& 흡입기 등' '낫' '니퍼'
 '다목적 가위' '다목적 드릴비트(육각)' '다목적가위' '다목적드릴비트세트' '다용도 공구세트' '다용도 미니 조명 '
 '단일지향 미니마이크 ' '드라이버 비트' '드라이버 십자 단용 8+' '드라이버비트' '드릴기리세트' '디지털염도계' '라돈기'
 '라돈측정기' '랜턴' '레이저줄자' '레이져거리측정기 ' '로터리햄머드릴' '롱 노우즈' '룩스패드43 203스탠드 ' '리드선'
 '리튬이온 밧데리' '리튬이온 배터리 충전기' '릴호스(20M)' '만능작업대' '망치' '목발(대/중/소)' '몽키 스패너'
 '몽키스패너' '무선전동 드라이버' '물호스' '미러리스카메라' '발 펌프' '발펌프' '방구석 방송세트' '방구석라이브세트'
 '방역분무기' '방역소독기' '배수구 청소기' '배터리충전기' '벡터 스트리퍼' '변기 관통기' '보쉬 GLM 40 거리측정기'
 '보쉬 수공구' '보쉬 열풍기' '보쉬수공구세트' '보안경' '보안경(물안경 타입)' '보행보조기' '볼렌치세트(8피스)'
 '볼렌치세트(9피스)' '볼트캇타' '부탄가스토치' '부탄가스히타' '부탄가스히터' '불

In [18]:
for i in df['cluster'].unique():
    print("공구 중분류:", i)
    print("개수 : ", len(df[df['cluster']==i]['공구 이름'].unique()))
    print(df[df['cluster']==i]['공구 이름'].unique())
    print("*"*50)
    # break
    

공구 중분류: -1
개수 :  2405
[' 고압세척기 ' '(57)레이저거리측정기' '4단사다리' ... '핸드드릴 1개' '햄머드릴 1개' '햄머드릴세트']
**************************************************
공구 중분류: 122
개수 :  10
['2단 사다리' '사다리(2단)' '4단 사다리' '사다리(2*4단)' '사다리(4*5단)' '사다리(4,5단)' '사다리(4단)'
 '사다리(A형 4단)' '사다리(광폭2단)' '사다리4단']
**************************************************
공구 중분류: 121
개수 :  7
['3단사다리' '3단 사다리' '가정사다리 3단' '사다리(3단)' '사다리-3단' '사다리3단' '접이형 사다리 3단']
**************************************************
공구 중분류: 6
개수 :  3
['4방향 냉동라쳇' '냉동라쳇' '4방항 냉동라쳇']
**************************************************
공구 중분류: 71
개수 :  9
['LED 작업등' 'led 작업등' '작업등' '작업등(충전)' '작업용LED' 'LED작업등' 'LED작업등(충전용)'
 'LED충전작업등' '충전식 작업등']
**************************************************
공구 중분류: 90
개수 :  5
['LED랜턴' '랜턴' 'LED 랜턴' '캠핑랜턴' '중전랜턴']
**************************************************
공구 중분류: 5
개수 :  3
['가스토치' '부탄가스토치' '가스토치(휴대용) 1개']
**************************************************
공구 중분류: 215
개수 :  6
['가정용 공구세트' '가정용 공구세트 ' '가정용공구세트' '가정용일반공구세트

In [16]:
class OllamaSentenceTransformer():
    def __inti__(
            self,
            *args,
            **kargs,
            ) -> None:
                self.base_url = kargs.get()
                self.model = kargs.get("model","bge-m3")
                self.embedding_model = OllamaEmbeddings(
                            model = self.model,
                            base_url = self.base_rul,
                        )
                        
    
    def encode(
            self,
            documents:Union[str, List[str], np.ndarray],
            *args,
            **kargs,
        )-> np.ndarray:
        if isinstance(documents, str):
            document_embeddings = self.embedding_model.embed_query(documents)
            return np.array(document_embeddings)
        
        if isinstance(documents, np.ndarray):
            documents = documents.tolist()
        
        document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
        return np.array(document_embeddings)
        
sentence_transformer = OllamaSentenceTransformer()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.551634,0.022563,-1.570104,-0.000510,0.740294,0.490988,-0.686420,-0.324609,-0.290186,-0.018750,...,-1.447591,-1.275745,-0.513019,0.683011,-0.693904,-0.084678,0.363890,0.561792,-0.527530,0.743740
1,0.225699,-0.024166,-0.923894,-0.797282,-0.402984,-0.535617,-0.362017,0.697372,0.992576,-0.052498,...,-0.467080,0.179963,-0.281391,0.729216,-0.770926,0.719807,1.748108,-0.886316,-0.192336,0.512210
2,-0.153459,0.760803,-0.972662,-0.188149,-0.298962,1.343238,1.205696,0.945845,-0.364392,-0.385127,...,0.220576,0.310735,0.617413,-0.437350,-0.672341,-0.295130,0.962149,0.001975,-0.030692,1.307994
3,0.125587,0.561656,-1.209939,-1.373026,-0.137662,2.486985,1.549750,0.914189,-1.010290,-0.914773,...,0.011740,-0.895031,0.176734,-0.791744,-0.479467,0.278657,1.475451,0.104340,0.078258,1.656066
4,1.327980,1.588254,-0.533199,-0.529462,0.147752,1.073482,1.515046,0.483241,-0.505042,-0.469459,...,0.497041,0.134544,-0.060856,-0.744806,-0.285829,0.454247,1.540792,-0.585578,0.149093,1.572348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11493,-1.177464,0.430387,-0.650017,-0.653407,-0.285039,-0.813194,0.038119,1.147779,0.156168,-0.804640,...,-1.511963,-0.830747,-0.373449,0.044326,-0.850673,0.058010,1.062544,0.756152,-1.315706,0.624219
11494,-0.854498,-0.483694,-0.512511,-0.384225,-0.193609,-0.663475,0.152227,0.579778,0.575798,-0.564336,...,-0.606831,-1.281012,-0.057233,0.332979,-0.724416,-0.751735,0.845496,0.630465,-1.772907,1.235120
11495,-0.854498,-0.483694,-0.512511,-0.384225,-0.193609,-0.663475,0.152227,0.579778,0.575798,-0.564336,...,-0.606831,-1.281012,-0.057233,0.332979,-0.724416,-0.751735,0.845496,0.630465,-1.772907,1.235120
11496,-0.854498,-0.483694,-0.512511,-0.384225,-0.193609,-0.663475,0.152227,0.579778,0.575798,-0.564336,...,-0.606831,-1.281012,-0.057233,0.332979,-0.724416,-0.751735,0.845496,0.630465,-1.772907,1.235120


In [1]:
df['공구 이름']

NameError: name 'df' is not defined

# Location Info 

In [5]:
import requests

def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '강남역'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


강남역 위도/경도: (37.4999282, 127.0263182)


In [ ]:
key = open('../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

In [29]:
client = OpenAI(api_key=api_key)
def invoke(question, info = user_info, model="gpt-4o", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages = [
                    {"role": "system", 
                        "content": f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                        너는 서울시에서 운영하는 대여 공구 찾기 정보 시스템이야. 
                        관련 정보는 {user_info}와 같아. 
                        이정보를 통해서 사용자의 답변에 친절해 답해줘.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ],
        temperature=temperature
    )
    return response.choices[0].message.content

In [35]:
[x for x in dir(np) if 'log' in x]

['log',
 'log10',
 'log1p',
 'log2',
 'logaddexp',
 'logaddexp2',
 'logical_and',
 'logical_not',
 'logical_or',
 'logical_xor',
 'logspace']

In [31]:
user_info

{0: {'구분자(PK)': 13423,
  '공구 대분류 코드': 4,
  '공구 대분류': '기타공구',
  '공구 중분류 코드': 44,
  '공구 중분류': '기타공구',
  '공구 이름': ' 고압세척기 ',
  '과금기준': '없음',
  '수량': 1,
  '장소 PK': 501,
  '대여장소명': '황학동 공구대여소 ',
  '상세주소': '서울특별시 중구 황학동 1010',
  '위도': 37.56742,
  '경도': 127.02137,
  '전화번호': '02-3396-6909',
  '평일오픈시간': 1000.0,
  '평일클로즈시간': 1700.0,
  '생성일시': Timestamp('2022-06-14 11:29:16'),
  '요금': '무료'},
 1: {'구분자(PK)': 14913,
  '공구 대분류 코드': 4,
  '공구 대분류': '기타공구',
  '공구 중분류 코드': 44,
  '공구 중분류': '기타공구',
  '공구 이름': '(57)레이저거리측정기',
  '과금기준': '없음',
  '수량': 1,
  '장소 PK': 172,
  '대여장소명': '우리마을 공구대여소(사당1동)',
  '상세주소': '서울 동작구 동작대로17길 28',
  '위도': 37.4830324,
  '경도': 126.9765661,
  '전화번호': '02-820-2571',
  '평일오픈시간': 900.0,
  '평일클로즈시간': 1800.0,
  '생성일시': Timestamp('2022-09-21 15:47:07'),
  '요금': '무료'},
 2: {'구분자(PK)': 14038,
  '공구 대분류 코드': 4,
  '공구 대분류': '기타공구',
  '공구 중분류 코드': 44,
  '공구 중분류': '기타공구',
  '공구 이름': '2단 사다리',
  '과금기준': '없음',
  '수량': 1,
  '장소 PK': 13214,
  '대여장소명': '묵1동 공유마을 공유대여소',
  '상세주소': '공릉로2가길 5',
  

In [32]:
question = f"나는 고압세척기를 빌리고싶어. 내가 사는 곳은 황학동이야. 근처에 있는 대여 장소가 어디 있는지 알려줘"

In [33]:
answer = invoke(question)
print("💬 GPT 응답:", answer)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 2656846 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [13]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야해.
    """
    response = client.responses.create(
        model='gpt-4.1',
        messages=[
                {"role": "system", "content": "당신은 완벽한 도우미입니다."},
                {"role": "user", "content": prompt}
            ],
        input = question
        )
    return response

In [14]:
output = invoke('배가 고픈데 생선을 먹고싶어. 6월에 먹을 수 있는 제철 생선 있어?')

TypeError: Responses.create() got an unexpected keyword argument 'messages'

In [20]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야돼
    """
    response = openai.ChatCompletion.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "당신은 완벽한 도우미입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    result_text = response['choices'][0]['message']['content']

    try:
        result_dict = json.loads(result_text)
        with open("clustered_equipment.json", "w", encoding="utf-8") as f:
            json.dump(result_dict, f, ensure_ascii=False, indent=2)
        print("✅ 클러스터링 결과 저장 완료: clustered_equipment.json")
        return result_dict
    except json.JSONDecodeError:
        print("⚠️ JSON 형식 오류. 출력 확인 필요:")
        print(result_text)
        return None
    

In [19]:
question = '나 너무 배고파'
result = invoke(question)

# 만약 JSON 결과를 바로 보고 싶으면
import pandas as pd

# if result:
#     df_result = pd.DataFrame(result)
#     display(df_result)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
prompt = f"""
    너는 도구가 어디있는지 찾아주는 도우미야. 
    사용자가 필요한 도구와 현재 위치를 알려주거야.
    너는 {df}를 참고해서 **사용자가 필요로 하는 도구**와 **현재 위치**에서 가장 가까이에있는 대여소를 찾아주면 돼.
    ============================
    """